In [183]:
import numpy as np
import pandas as pd

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 80)

import datetime
from datetime import datetime, date

### Сырые данные по хакатону

In [185]:
# Читаем файл сырых данных
hackaton_file = '/Users/op/dev/olgapavlova/agile-health-hackathon/csv/data_for_spb_hakaton_entities1-Table 1.csv'
dhack = pd.read_csv(hackaton_file,
                    header=1,               # в первой строчке мусор, во второй — названия колонок
                    index_col='entity_id',  # ключ в первой колонке
                    dtype = {
                                'parent_ticket_id': 'Int64',
                                'estimation': 'Int64',
                                'spent': 'Int64'
                            },              # есть незаполненные parent_ticket_id, поэтому Int64, не int
                    parse_dates = [
                                    'create_date',
                                    'update_date'
                                  ],        # в этих колонках легко распознаваемые даты (и всегда есть)
                    delimiter=';')          # нетипичный csv

#### Все колонки сырых данных по хакатону

In [187]:
dhack.dtypes

area          object
type          object
status        object
state         object
priority      object
               ...  
rank          object
estimation     Int64
spent          Int64
workgroup     object
resolution    object
Length: 20, dtype: object

#### Команда
* [ ] По командам нужно уметь фильтровать.
* [ ] Значения явно взяты из словаря. В нашей базе данных стоит воспроизвести этот же словарь отдельной таблицей. Заодно получим ID значения, а не само это значение (что удобней).

In [189]:
# Сколько каких команд в системе
dhack.groupby('area').size()

area
Система. Движок                    264
Система.Вики                       560
Система.Ошибки                      51
Система.Таск-трекер                761
Система.ХранениеАртефактов         650
Управление релизами изменениями    199
dtype: int64

* [ ] В интерфейсе нет смысла использовать значения сырья — страшные. Лучше почистить.
* [ ] Идеально было бы сделать конфигуратор этого переименования прямо в интерфейсе.

In [191]:
replace_area = {
    "Система. Движок": "Движок",
    "Система.Вики": "Wiki",
    "Система.Ошибки": "Ошибки",
    "Система.Таск-трекер": "Task tracker",
    "Система.ХранениеАртефактов": "Хранение",
    "Управление релизами изменениями": "Релизы",
    }
dhack['area'] = dhack['area'].replace(replace_area)
dhack.groupby('area').size()

area
Task tracker    761
Wiki            560
Движок          264
Ошибки           51
Релизы          199
Хранение        650
dtype: int64

#### Тип задачи

Тоже словарь. Видимо, схема простая: История → Задача → Подзадача.

In [194]:
dhack.groupby('type').size()

type
Дефект        654
Задача       1021
История        55
Подзадача     755
dtype: int64

#### Статусы задач
* Затейливо. Очень. Схема: https://github.com/olgapavlova/agile-health-hackathon/blob/main/WORKFLOW.md
* Задачи делятся на те, что замораживают ресурс («в производстве») и те, что уже приносят пользу («выпущенные»).
* Очевидно есть две группы производственных задач: «Работа идёт» и «Работа стоит». Их надо показывать по-разному.
* Некоторые статусы, возможно, стоит объединять. Это тоже гибкая настройка.

In [196]:
all_statuses = dhack.groupby('status').size().sort_values()
pd.set_option('display.max_rows', len(all_statuses))
all_statuses
# pd.reset_option('display.max_rows')

status
В ожидании                      1
Готово к разработке             1
СТ Завершено                    2
Локализация                     3
Подтверждение                   3
СТ                              4
Разработка                      5
Отложен                         7
Исправление                    11
Анализ                         12
Подтверждение исправления      15
Тестирование                   19
Отклонен исполнителем          20
В работе                       33
Создано                        90
Выполнено                     157
Закрыто                      2102
dtype: int64

#### Состояние
Везде одно и то же значение. Игнорируем столбец.

In [198]:
dhack['state'].unique()

array(['Normal'], dtype=object)

#### Приоритет
* [ ] Очевидно, нужен упорядоченный словарь.
* [ ] Пригодится колонка «Вес» для разных хитрых подсчётов.

In [200]:
dhack['priority'].unique()

array(['Средний', 'Критический', 'Высокий', 'Низкий'], dtype=object)

#### Номер тикета
* В буквах тикета (первые четыре знака, до дефиса) закодирована команда (area). Пригодится в интерфейсе.

In [202]:
ticket_research = dhack[['ticket_number', 'area']]
ticket_research.head(20)

          ticket_number          area
entity_id                            
94297         PPTS-1965  Task tracker
102481        PPIN-1175        Ошибки
1805925       PPTS-3189  Task tracker
1934905       PPTS-3383  Task tracker
1943849       PPIN-1609        Ошибки
...                 ...           ...
2824925       PPIN-2444        Ошибки
2840563       PPTS-4895  Task tracker
2867204       PPFW-1314        Движок
2887761       PPAR-4466      Хранение
2938657       PPTS-5013  Task tracker

[20 rows x 2 columns]

#### Название тикета
* В названии чего только нет! Разобраться.
* Пустота в начале что-нибудь значит?

In [204]:
dhack['create_date'].head()

entity_id
94297     2023-03-16 16:59:00.000000
102481    2023-05-12 13:33:55.918127
1805925   2023-07-12 09:36:04.479760
1934905   2023-08-04 11:32:25.829919
1943849   2023-08-09 06:20:44.391950
Name: create_date, dtype: datetime64[ns]

In [205]:
pd.set_option('display.max_colwidth', None)
dhack['name'].head(20)

entity_id
94297                                    [FE] Бэклог. Кастомизация колонок. Кастомизация для панелей "спринты" и  "бэклоги" зависима друг от друга
102481                                   [ГенераторДокументов] Интеграция со Система.ГенераторДокументов в части синхронизации Стандартных решений
1805925          [FE] История изменений. Пустые строки в истории изменений, если выбрать значение, которое уже было выбрано до редактирования поля
1934905                                                             [FE] Зависимые поля. Тип реакции disable НЕ работает для некоторых типов полей
1943849                    [BE] При сортировке по Теме если знаки препинания стоят перед буквенными значениями сортировка отрабатывает неправильно
                                                                            ...                                                                   
2824925                                                                                      [ML Инц-2] Груп

#### Кем создан
* Не так уж много вариантов. Но кто эти люди — непонятно. Хорошо бы уметь настраивать имена.
* Возможно, к задачам особо активных нужно проявить особое отношение в интерфейсе?
* Только кириллица. Это приятно.

In [207]:
created_by_research = dhack.groupby('created_by').size()
pd.set_option('display.max_rows', len(created_by_research))
created_by_research.sort_values()

created_by
Н. М.      1
М. А.      1
Л. П.      1
М. С.      1
С. Т.      1
Д. Н.      1
К. У.      1
С. А.      1
В. Т.      1
И. Ш.      1
В. Л.      1
Я. Е.      2
А. Л.      2
А. Ж.      2
А. Е.      2
Т. Т.      2
П. Л.      2
В. И.      2
С. Б.      2
Н. Л.      3
И. П.      3
Д. З.      3
А. У.      3
В. Ш.      3
Е. Ш.      3
С. Н.      3
А. Г.      3
М. Т.      3
Е. Л.      4
Е. К.      4
И. С.      4
Р. Р.      4
П. С.      5
М. Ф.      7
Т. С.      7
В. К.      7
П. М.      8
О. У.      9
Е. В.      9
В. П.     10
Т. Ч.     10
Н. Т.     11
Д. Ш.     11
А. Б.     11
К. Я.     11
М. Н.     12
Д. В.     12
В. Ц.     12
А. И.     12
А. Ш.     13
И. Д.     14
О. Н.     15
В. Д.     15
Н. К.     15
П. Х.     17
М. Я.     17
Д. Г.     18
Е. М.     20
А. Д.     20
Н. В.     20
В. Р.     23
И. Т.     24
И. М.     24
А. Я.     25
А. С.     26
В. М.     27
О. Г.     27
Р. Б.     29
Р. Е.     35
М. К.     40
А. З.     41
А. Т.     41
О. К.     42
В. С.     42
И. К.     51
Д. Б.     56
М

#### Дата последнего обновления
* Стандартная дата, легко парсится, ничего особенного.

In [209]:
dhack['update_date'].head(20)

entity_id
94297     2024-09-10 11:20:09.193785
102481    2024-08-06 19:30:16.692683
1805925   2024-11-05 15:02:00.900484
1934905   2024-08-13 11:46:20.165757
1943849   2024-07-18 08:23:39.657571
1966759   2024-09-04 07:17:56.613513
2353124   2024-09-19 15:48:16.634504
2383243   2024-07-30 15:07:44.863470
2441405   2024-09-03 10:33:50.369963
2444222   2024-07-16 12:06:16.525857
2503655   2024-07-22 13:10:34.908571
2536878   2024-08-06 05:10:40.901890
2540151   2024-09-11 08:29:51.809354
2582366   2024-10-25 09:15:45.884875
2623949   2024-07-31 09:22:53.938131
2824925   2024-08-06 19:30:16.710853
2840563   2024-09-02 11:41:38.380514
2867204   2024-07-10 08:10:46.949956
2887761   2024-07-03 14:32:01.022170
2938657   2024-09-09 12:51:53.466795
Name: update_date, dtype: datetime64[ns]

#### Кто обновлял в последний раз
* Где-то кириллица, а где-то латиница. Обращает на себя внимание.

In [211]:
updated_by_research = dhack.groupby('updated_by').size()
pd.set_option('display.max_rows', len(updated_by_research))
updated_by_research.sort_values()

updated_by
A. U.      1
Д. У.      1
Д. К.      1
К. Я.      1
Е. П.      1
Р. Г.      1
С. Т.      1
Е. М.      1
S. T.      1
Y. K.      1
Ш. Г.      1
V. V.      1
В. Ч.      2
П. С.      2
А. Л.      2
А. Р.      3
С. Б.      3
А. Б.      3
Ю. М.      3
Р. А.      4
М. Н.      4
Д. Н.      6
И. Ч.      6
А. О.      6
И. Д.      6
Р. Р.      6
Т. К.      6
В. Ц.      6
В. Ш.      6
В. Ю.      7
М. А.      7
Н. М.      7
И. П.      7
Д. Ш.      7
С. Н.      7
И. Ш.      7
Я. Е.      8
С. А.      8
К. Г.      8
Т. С.      9
Р. М.      9
М. Т.      9
М. Ф.     10
О. У.     10
Д. Ф.     10
Т. Т.     10
Р. В.     11
Р. Б.     12
П. Х.     13
Е. Ш.     13
Д. З.     13
А. Ш.     14
Е. К.     15
Т. Х.     15
Д. В.     16
И. С.     16
А. И.     16
Е. В.     16
М. К.     17
А. Д.     18
А. Е.     19
В. Р.     19
М. Я.     19
Д. Г.     20
В. П.     20
Ф. Л.     21
О. Г.     22
И. Т.     23
А. З.     23
Н. В.     23
Т. Ч.     24
А. С.     24
А. У.     25
В. Д.     26
Е. З.     28
В. К.     29
П

#### Родительский тикет
* Целые числа. По умолчанию в Dataframe появляется дробная часть :)
* Есть тикеты-матки. Без родителей.
* Прикольно было бы что-то сделать с деревом тикетов, конечно.

In [213]:
dhack['parent_ticket_id'].head()

entity_id
94297        72779
102481     3488105
1805925       <NA>
1934905       <NA>
1943849    2083371
Name: parent_ticket_id, dtype: Int64

In [214]:
# Сколько тикетов без родителей
dhack['parent_ticket_id'].isnull().sum()

807

#### Кто ответственный за тикет
* Тоже местами всплывает латиница. Скорее всего, это ошибка, и надо дать пользователю возможность её исправить — назначить синонимы.

In [216]:
dhack.groupby('assignee').size().sort_values()

assignee
S. T.      1
V. E.      1
V. V.      1
Ш. Г.      1
В. И.      1
Д. З.      1
Д. К.      1
С. Т.      1
И. Н.      1
В. Ц.      1
С. Ш.      1
A. U.      2
В. Л.      2
А. О.      2
И. М.      2
Л. П.      2
С. Б.      3
Д. У.      3
А. Е.      4
Р. Б.      4
Р. А.      4
А. Л.      4
В. Р.      5
И. Ч.      6
В. Ч.      6
Н. М.      6
К. Г.      7
А. Р.      7
Я. Е.      7
И. Ш.      8
И. Д.      8
В. Ш.      8
А. Ж.      8
Р. Р.      9
Д. Ш.      9
М. Т.     10
М. Ф.     10
М. А.     10
М. К.     11
Т. Т.     11
К. Я.     12
Р. В.     13
Е. В.     13
Т. С.     14
А. Я.     14
Е. З.     14
А. Б.     14
М. Я.     15
Е. Ш.     16
Р. Е.     16
А. И.     16
Д. С.     16
П. Х.     17
И. П.     17
О. К.     18
Д. Н.     19
Д. Г.     19
В. Ю.     19
А. З.     19
А. Ш.     19
А. Т.     20
О. Г.     20
И. С.     21
И. Б.     23
В. Д.     23
С. Н.     24
Т. Х.     24
С. А.     24
В. П.     24
Е. К.     26
Ф. Л.     27
Д. В.     28
М. Б.     28
Н. В.     28
Е. Б.     30
А. У.     31
П. 

#### Кто владелец тикета
* Неожиданно много.
* Надо проверить, совпадает ли владелец с назначившим.

In [218]:
dhack.groupby('owner').size().sort_values()

owner
Л. П.      1
И. Б.      1
И. Ш.      1
К. У.      1
Д. У.      1
М. А.      1
М. В.      1
М. С.      1
Д. Н.      1
Р. Е.      1
П. Х.      1
А. Л.      2
И. П.      2
А. Г.      2
В. И.      2
А. Р.      2
А. Ж.      3
Н. Л.      3
М. Т.      3
В. К.      3
И. К.      3
В. П.      3
С. Н.      3
Е. В.      3
А. Е.      3
Е. Ш.      3
Т. Ч.      4
М. Ф.      4
Е. Л.      4
С. Т.      4
А. И.      4
Д. З.      5
С. Б.      6
Т. С.      7
П. С.      7
М. Ч.      7
В. Д.      8
С. А.      8
О. У.     10
Н. Т.     11
А. Ш.     11
В. Ц.     12
К. Я.     12
Д. Г.     12
Д. Ш.     12
А. Б.     12
М. Н.     13
Н. В.     14
О. Н.     14
И. Д.     15
О. Г.     16
А. Д.     16
Е. М.     19
Н. К.     21
А. С.     23
Д. Б.     24
О. К.     28
В. М.     29
В. С.     34
А. Т.     35
А. З.     35
Д. П.     36
В. Р.     38
М. Я.     38
И. Т.     40
М. Б.     42
Р. Б.     48
М. Д.     57
Б. П.     57
Н. Н.     59
А. М.     63
М. К.     64
Е. Б.     74
Д. С.     79
Е. З.     82
А. П.     86
И. М. 

#### Какая-то дата (исполнения?)
* В чём смысл этой колонки? Что-то про дату.
* Почти всё пустое. Игнорим?

In [220]:
dhack[['due_date', 'name']].sort_values(by='due_date').head(10)

           due_date  \
entity_id             
4498695    10/22/24   
4514809    10/31/24   
4373318    10/31/24   
4594769    10/31/24   
4491114    10/31/24   
4914666    10/31/24   
5172513     10/9/24   
5172290     10/9/24   
4603898     11/7/24   
4309869     6/27/24   

                                                                                                                      name  
entity_id                                                                                                                   
4498695                                                                          [PPPL] Разработка документации по BE SDK   
4514809       [Дефект][ПРОМ][FE] не выделяются инлайн комментарии и не отображаются комментарии к ним на старых страницах   
4373318                                          [Дефект][ПРОМ][FE] не отображается макрос QL в макросе "вложенная статья"  
4594769                          [Дефект][ПРОМ][FE] Невидимая кнопка "скачать" при открытии картин

In [221]:
dhack['rank'].head()

entity_id
94297                                                                                                                                                                                                                                                                                                                              0|qzzywk:
102481                                                                                                                                                                                                                                                                                                                            0|qv7n1c:y
1805925                                                                                                                                                                                                                                                                                                                            0

In [222]:
# Считаем пустые ячейки
dhack['due_date'].isna().sum()

2431

#### Ранг?
* Что это значит? Какой-то внутренний код? Как можем использовать?
* Есть ошибки в данных.

#### Оценка трудозатрат
* Дана в секундах, что ли?
* Делим на 3600, чтобы получить часы.

In [225]:
dhack['estimation'].head()

entity_id
94297          60
102481     432000
1805925        60
1934905      <NA>
1943849      <NA>
Name: estimation, dtype: Int64

In [226]:
estimation_hours = (dhack['estimation'].fillna(0) / 3600).astype('Int64')
estimation_hours.sort_values(ascending=False).head(10)

entity_id
3725698    264
4455271    240
4012062    160
4317974    152
102481     120
4510102    120
4441120     96
4817393     80
4793198     80
4319042     80
Name: estimation, dtype: Int64

#### Уже потраченное время
* Так же, как и оценкой — делим на 3600.

In [228]:
spent_hours = (dhack['spent'].fillna(0) / 3600).astype('Int64')
spent_hours.sort_values(ascending=False).head()

entity_id
5005327    80
4861719    64
4558467    64
4438203    64
4444351    64
Name: spent, dtype: Int64

#### Рабочая группа
* Что это?
* Называется вроде осмысленно, можно не править.
* Много пустых ячеек.

In [230]:
dhack.groupby('workgroup').size().sort_values()

workgroup
Архитектурная задача       15
Технический долг           66
Линейная деятельность     273
Новая функциональность    722
dtype: int64

In [231]:
# Считаем пустые ячейки
dhack['workgroup'].isna().sum()

1409

#### Резолюция
* Есть некоторое количество пустых. Что с ними делать?

In [233]:
dhack.groupby('resolution').size()

resolution
Готово                 1926
Дубликат                 23
Отклонено                33
Отменен инициатором      45
dtype: int64

In [234]:
# Считаем пустые ячейки
dhack['resolution'].isna().sum()

458

## Архив

In [236]:
#dhis = pd.read_csv(history_file, delimiter=';')
#dser = pd.read_csv(series_file, delimiter=';')